In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import random

In [2]:
# Function to select [size] random rows and set a random feature on them to null
def add_errors(data: pd.DataFrame, size: int) -> pd.DataFrame:
    indexes = np.random.randint(0, len(data) - 1, size)
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=FutureWarning)    
        columns = ['alcohol', 'pH', 'chlorides'] 
        for i in indexes:
            column = np.random.choice(columns)
            error_type = np.random.choice(['missing', 'out_of_range'])  
            
            if error_type == 'missing':
                data.loc[i, column] = np.NaN
            
            elif error_type == 'out_of_range': 
                if column == 'alcohol':
                    data.loc[i, column] = np.random.uniform(15, 40)
                elif column == 'pH':
                    data.loc[i, column] = np.random.uniform(8, 10)
                elif column == 'chlorides':
                    data.loc[i, column] = np.random.uniform(2, 4)      

    print(data.isnull().sum().sum(), 'null values created in data')
    return data


In [3]:
# Function to introduce missing columns in randomly selected files
def introduce_missing_columns(num_files: int) -> None:
    raw_data_files = os.listdir("../raw_data")
    files_to_modify = np.random.choice(raw_data_files, num_files, replace=False)
    
    columns = ['fixed acidity', 'volatile acidity']
    
    for file in files_to_modify:
        df = pd.read_csv(f'../raw_data/{file}')
        columns_to_drop = random.sample(columns, random.choice([1,2]))
        print(f"Dropping columns {columns_to_drop} from file {file}")
        df.drop(columns_to_drop, axis=1, inplace=True)
        df.to_csv(f'../raw_data/{file}', index=False)


In [4]:
# Function to split one file into three smaller files
def split_one_file_into_four() -> None:
    raw_data_files = os.listdir("../raw_data")
    files = np.random.choice(raw_data_files, 2)
    
    for file_to_split in files:
        df = pd.read_csv(f'../raw_data/{file_to_split}')
        chunks = np.array_split(df, 4)
        print(f"Splitting file {file_to_split} into 4 files.")
        for i, chunk in enumerate(chunks):
            chunk.to_csv(f'../raw_data/data_chunk_{len(raw_data_files) + i + 1}.csv', index=False)


In [5]:
# Function to repeat rows in the dataset
def repeat_row_in_files(num_files: int) -> None:
    raw_data_files = os.listdir("../raw_data")
    files_to_modify = random.sample(raw_data_files, min(num_files, len(raw_data_files)))
    
    for file in files_to_modify:
        df = pd.read_csv(f'../raw_data/{file}')
        row_to_repeat = df.sample(1)
        print(f"Repeating row {row_to_repeat['index']} in file {file}")
        df = pd.concat([df, row_to_repeat], ignore_index=True)
        df.to_csv(f'../raw_data/{file}', index=False)


In [6]:
# Function to split and save data
def split_and_save_data(data: pd.DataFrame, num_files: int) -> None:
    total_rows = len(data)
    rows_per_file = total_rows // num_files
    remaining_rows = total_rows % num_files
    
    for _ in range(1, num_files + 1):
        if remaining_rows > 0:
            chunk_size = rows_per_file + 1
            remaining_rows -= 1
        else:
            chunk_size = rows_per_file
        
        chunk = data.sample(n=min(chunk_size, len(data)))
        filename = f'data_chunk_{len(os.listdir("../raw_data")) + 1}.csv'
        
        chunk.to_csv(f'../raw_data/{filename}', index=False)
        data = data.drop(chunk.index)
        
        if data.empty:
            print("DataFrame is now empty. Exiting...")
            break


In [7]:
df = pd.read_csv('../data/winequality-red.csv')
df = df.reset_index()

# Introduce NaN and out of range errors
df_with_errors = add_errors(df.copy(), 200)

num_files = 100  # Specify the number of files to create
split_and_save_data(df_with_errors, num_files)

introduce_missing_columns(15)  # Introduce missing columns in 5 random files
split_one_file_into_four()
repeat_row_in_files(15)  # Repeat random rows in 5 files


98 null values created in data
DataFrame is now empty. Exiting...
Dropping columns ['volatile acidity'] from file data_chunk_15.csv
Dropping columns ['fixed acidity'] from file data_chunk_16.csv
Dropping columns ['fixed acidity', 'volatile acidity'] from file data_chunk_39.csv
Dropping columns ['volatile acidity'] from file data_chunk_96.csv
Dropping columns ['volatile acidity'] from file data_chunk_19.csv
Dropping columns ['volatile acidity'] from file data_chunk_74.csv
Dropping columns ['volatile acidity'] from file data_chunk_58.csv
Dropping columns ['volatile acidity', 'fixed acidity'] from file data_chunk_9.csv
Dropping columns ['fixed acidity'] from file data_chunk_45.csv
Dropping columns ['volatile acidity'] from file data_chunk_56.csv
Dropping columns ['fixed acidity'] from file data_chunk_55.csv
Dropping columns ['fixed acidity'] from file data_chunk_5.csv
Dropping columns ['fixed acidity', 'volatile acidity'] from file data_chunk_14.csv
Dropping columns ['volatile acidity'] f

c:\Users\bemne\anaconda3\envs\dsp-project\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
c:\Users\bemne\anaconda3\envs\dsp-project\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Splitting file data_chunk_3.csv into 4 files.
Splitting file data_chunk_72.csv into 4 files.
Repeating row 0    382
Name: index, dtype: int64 in file data_chunk_102.csv
Repeating row 6    304
Name: index, dtype: int64 in file data_chunk_2.csv
Repeating row 10    1566
Name: index, dtype: int64 in file data_chunk_32.csv
Repeating row 6    1179
Name: index, dtype: int64 in file data_chunk_60.csv
Repeating row 10    834
Name: index, dtype: int64 in file data_chunk_93.csv
Repeating row 8    1221
Name: index, dtype: int64 in file data_chunk_95.csv
Repeating row 3    89
Name: index, dtype: int64 in file data_chunk_36.csv
Repeating row 8    1008
Name: index, dtype: int64 in file data_chunk_19.csv
Repeating row 8    58
Name: index, dtype: int64 in file data_chunk_35.csv
Repeating row 11    390
Name: index, dtype: int64 in file data_chunk_43.csv
Repeating row 1    437
Name: index, dtype: int64 in file data_chunk_88.csv
Repeating row 5    279
Name: index, dtype: int64 in file data_chunk_67.csv
Re